In [2]:
import numpy as np

In [3]:
max_word_length = 50 #최대 단어의 자수: 50자 
_max_word_length = max_word_length # 50

In [4]:
# char ids 0-255 come from utf-8 encoding bytes
# assign 256-300 to special chars
bos_char = 256  # <begin sentence>
eos_char = 257  # <end sentence>

bow_char = 258  # <begin word>
eow_char = 259  # <end word>

pad_char = 260 # <padding>

In [5]:
num_words = 793471 #단어사전의 총 단어 개수

In [6]:
_word_char_ids = np.zeros([num_words, max_word_length], #793471, 50 
  dtype=np.int32)
_word_char_ids.shape # (100, 50) -> 100개 단어, 각 단어 자수: 최대 50자

(793471, 50)

In [7]:
 # the charcter representation of the begin/end of sentence characters
def _make_bos_eos(c):
    r = np.zeros([max_word_length], dtype=np.int32) #최대 50문자
    r[:] = pad_char 
    r[0] = bow_char # <begin word> -> 문자의 시작
    r[1] = c
    r[2] = eow_char # <end word> -> 문자의 끝
    return r

In [8]:
print("bos_char:", bos_char)
bos_chars = _make_bos_eos(bos_char)
print("bos_chars:", bos_chars)

bos_char: 256
bos_chars: [258 256 259 260 260 260 260 260 260 260 260 260 260 260 260 260 260 260
 260 260 260 260 260 260 260 260 260 260 260 260 260 260 260 260 260 260
 260 260 260 260 260 260 260 260 260 260 260 260 260 260]


In [9]:
print("eos_char:", eos_char)
eos_chars = _make_bos_eos(eos_char)
print("eos_chars:", eos_chars)

eos_char: 257
eos_chars: [258 257 259 260 260 260 260 260 260 260 260 260 260 260 260 260 260 260
 260 260 260 260 260 260 260 260 260 260 260 260 260 260 260 260 260 260
 260 260 260 260 260 260 260 260 260 260 260 260 260 260]


In [20]:
#import urllib
import urllib.request

In [21]:
class Vocabulary(object):
    '''
    A token vocabulary.  Holds a map from token to ids and provides
    a method for encoding text to a sequence of ids.
    '''
    def __init__(self, filename, validate_file=False):
        '''
        filename = the vocabulary file.  It is a flat text file with one
            (normalized) token per line.  In addition, the file should also
            contain the special tokens <S>, </S>, <UNK> (case sensitive).
        '''
        self._id_to_word = []
        self._word_to_id = {}
        self._unk = -1 #word_name == '<UNK>'
        self._bos = -1 #word_name == '<S>'
        self._eos = -1 #word_name == '</S>'

        #with open(filename) as f:
            #idx = 0
            #for line in f:
        
        idx = 0
        url = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/vocab-2016-09-10.txt"
        file = urllib.request.urlopen(url)
        for line in file:
          line = line.decode("utf-8")
          word_name = line.strip()
          if word_name == '<S>':
              self._bos = idx
          elif word_name == '</S>':
              self._eos = idx
          elif word_name == '<UNK>':
              self._unk = idx
          if word_name == '!!!MAXTERMID':
              continue

          self._id_to_word.append(word_name)
          self._word_to_id[word_name] = idx
          idx += 1

        # check to ensure file has special tokens
        if validate_file:
            if self._bos == -1 or self._eos == -1 or self._unk == -1:
                raise ValueError("Ensure the vocabulary file has "
                                 "<S>, </S>, <UNK> tokens")

    @property
    def bos(self):
        return self._bos

    @property
    def eos(self):
        return self._eos

    @property
    def unk(self):
        return self._unk

    @property
    def size(self):
        return len(self._id_to_word)

    def word_to_id(self, word):
        if word in self._word_to_id:
            return self._word_to_id[word]
        return self.unk

    def id_to_word(self, cur_id):
        return self._id_to_word[cur_id]

    def decode(self, cur_ids):
        """Convert a list of ids to a sentence, with space inserted."""
        return ' '.join([self.id_to_word(cur_id) for cur_id in cur_ids])

    def encode(self, sentence, reverse=False, split=True):
        """Convert a sentence to a list of ids, with special tokens added.
        Sentence is a single string with tokens separated by whitespace.
        If reverse, then the sentence is assumed to be reversed, and
            this method will swap the BOS/EOS tokens appropriately."""

        if split:
            word_ids = [
                self.word_to_id(cur_word) for cur_word in sentence.split()
            ]
        else:
            word_ids = [self.word_to_id(cur_word) for cur_word in sentence]

        if reverse:
            return np.array([self.eos] + word_ids + [self.bos], dtype=np.int32)
        else:
            return np.array([self.bos] + word_ids + [self.eos], dtype=np.int32)

In [22]:
def test_urllib():
  url = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/vocab-2016-09-10.txt"
  file = urllib.request.urlopen(url)
  for line in file:
    decoded_line = line.decode("utf-8")
    print(decoded_line)
    break
test_urllib()

</S>



In [23]:
vocab_file = "no_file"
vocab = Vocabulary(vocab_file, validate_file=True)

In [24]:
print(vocab._id_to_word)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
print(len(vocab._id_to_word))

793471


In [26]:
print(vocab._word_to_id.keys())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
print(vocab._unk)
print(vocab._bos)
print(vocab._eos)

2
1
0


In [28]:
def _convert_word_to_char_ids(word):
    code = np.zeros([max_word_length], dtype=np.int32) #50
    code[:] = pad_char

    # utf-8로 인코딩된 word 
    word_encoded = word.encode('utf-8', 'ignore')[:(max_word_length-2)] #50-2=48
    #print("word_encoded:", word_encoded)
    
    code[0] = bow_char #begin of word -> 단어의 시작
    # word의 한 문자씩 넣음
    for k, chr_id in enumerate(word_encoded, start=1):
        code[k] = chr_id
    code[len(word_encoded) + 1] = eow_char #end of word -> 단어의 끝

    return code

In [29]:
for i, word in enumerate(vocab._id_to_word):
  #print("i:", i)
  #print("word:", word)
  _word_char_ids[i] = _convert_word_to_char_ids(word)
  #print("_word_char_ids[i]:", _word_char_ids[i])
  #if i > 5:
    #break

In [30]:
_word_char_ids.shape # (793471, 50)

(793471, 50)

In [31]:
_word_char_ids[0] # 한 단어씩 시작 

array([258,  60,  47,  83,  62, 259, 260, 260, 260, 260, 260, 260, 260,
       260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260,
       260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260,
       260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260], dtype=int32)

In [32]:
# 내부 접근을 외부 접근 가능 변수로 바꾸기 
#@property
#def bos(self):
  #return self._bos
vocab.bos

1

In [33]:
# bos 단어에 대한 charids값
# eos 단어에 대한 charids값
_word_char_ids[vocab.bos] = bos_chars
_word_char_ids[vocab.eos] = eos_chars